In [1]:
# Library Imports
import pickle
import streamlit as st # Version 1.25.0
import pandas as pd # Version 2.0.1
import numpy as np # Version 1.3.0
import seaborn as sns # Version 0.12.2
import matplotlib.pyplot as plt # version 3.6.3
from sklearn.ensemble import GradientBoostingRegressor # Version 1.3.0

In [4]:
# Read in Dat
data = pd.read_csv("../Data/model_ready.csv.gz", compression = "gzip")

# Load models
house_model = pickle.load(open("../Code Library/Models/gb_house_tuned.sav", "rb"))
room_model = pickle.load(open("../Code Library/Models/gb_room_tuned.sav", "rb"))

# Load fitted pipelines
house_pipeline = pickle.load(open("../Deployment_Files/house_pipeline.sav", "rb"))
room_pipeline = pickle.load(open("../Deployment_Files/room_pipeline.sav", "rb"))

In [5]:
data

,host_listings_count,property_type,room_type,bathrooms,bedrooms,price,minimum_nights,maximum_nights,minimum_minimum_nights,maximum_maximum_nights,...,calculated_host_listings_count,calculated_host_listings_count_private_rooms,calculated_host_listings_count_shared_rooms,reviews_per_month,zipcode,median_income_dollars,property_type_binary,private,sentiment,review_score_weighted
0,52.0,Entire home,Entire home/apt,2.0,2.0,728.0,2,28,1.0,1125.0,...,30,0,0,3.77,92109,95170.0,house,1,0.900577,150.04
1,0.0,Private room in condo,Private room,1.0,1.0,70.0,7,30,7.0,30.0,...,1,1,0,0.15,92109,95170.0,room,1,0.997769,5.00
2,4.0,Entire home,Entire home/apt,3.0,4.0,595.0,3,365,3.0,365.0,...,3,0,0,1.53,92109,95170.0,house,1,0.999661,45.00
3,0.0,Entire home,Entire home/apt,1.0,3.0,926.0,2,14,2.0,1125.0,...,2,0,0,0.00,92109,95170.0,house,1,0.999559,0.00
4,4.0,Camper/RV,Entire home/apt,1.0,1.0,62.0,1,1125,1.0,1125.0,...,3,1,0,0.12,92109,95170.0,house,0,0.062897,5.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18489,1.0,Entire rental unit,Entire home/apt,1.0,1.0,96.0,4,1125,4.0,1125.0,...,1,0,0,2.30,92020,64691.0,house,1,0.321106,748.50
18490,4.0,Entire home,Entire home/apt,2.0,3.0,262.0,1,365,1.0,365.0,...,3,0,0,1.00,92020,64691.0,house,1,0.844013,5.00
18491,1.0,Private room in home,Private room,1.0,1.0,50.0,2,30,2.0,30.0,...,1,1,0,0.53,92020,64691.0,room,1,0.999578,156.16
18492,1.0,Entire guesthouse,Entire home/apt,1.0,2.0,175.0,1,365,1.0,365.0,...,1,0,0,4.23,92020,64691.0,house,1,0.057779,50.00


In [14]:
tuple(data.room_type.values)

('Entire home/apt',
 'Private room',
 'Entire home/apt',
 'Entire home/apt',
 'Entire home/apt',
 'Entire home/apt',
 'Entire home/apt',
 'Entire home/apt',
 'Entire home/apt',
 'Entire home/apt',
 'Private room',
 'Private room',
 'Entire home/apt',
 'Entire home/apt',
 'Entire home/apt',
 'Entire home/apt',
 'Entire home/apt',
 'Private room',
 'Entire home/apt',
 'Entire home/apt',
 'Entire home/apt',
 'Entire home/apt',
 'Entire home/apt',
 'Entire home/apt',
 'Entire home/apt',
 'Entire home/apt',
 'Entire home/apt',
 'Entire home/apt',
 'Entire home/apt',
 'Entire home/apt',
 'Entire home/apt',
 'Entire home/apt',
 'Entire home/apt',
 'Entire home/apt',
 'Entire home/apt',
 'Entire home/apt',
 'Entire home/apt',
 'Entire home/apt',
 'Entire home/apt',
 'Entire home/apt',
 'Entire home/apt',
 'Entire home/apt',
 'Entire home/apt',
 'Entire home/apt',
 'Entire home/apt',
 'Private room',
 'Entire home/apt',
 'Entire home/apt',
 'Entire home/apt',
 'Entire home/apt',
 'Entire home/a